## Import dependencies

In [1]:
import sys
import os

sys.path.append("/home/jovyan/work/sem-covid/")
sys.path = list(set(sys.path))
os.getcwd()
os.chdir('/home/jovyan/work/sem-covid/')

import numpy as np
from gensim.models import KeyedVectors

from sem_covid.services.data_registry import Dataset, LanguageModel
from sem_covid.services.ml_flow_models import get_best_model_from_ml_flow
from sem_covid.services.sc_wrangling.mean_vectorizer import text_to_vector
import pandas as pd




/opt/conda/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


## Define consts

In [2]:
EU_TIMELINE_TEXT_COLUMNS = ['title', 'abstract', 'detail_content', 'detail_title']

EU_TIMELINE_NEW_COLUMNS = ['businesses', 'citizens','workers']

## Load dataset and language model

In [3]:
df = Dataset.EU_ACTION_TIMELINE.fetch()
law2vec = LanguageModel.LAW2VEC.fetch()
law2vec_path = LanguageModel.LAW2VEC.path_to_local_cache()
law2vec_format = KeyedVectors.load_word2vec_format(law2vec_path, encoding="utf-8")
l2v_dict = {w: vec for w, vec in zip(law2vec_format.index_to_key, law2vec_format.vectors)}

100% (171 of 171) |######################| Elapsed Time: 0:00:00 Time:  0:00:00


## Prepare dataset

In [4]:
df.loc[df['detail_title'].isnull(), 'detail_title'] = df.loc[df['detail_title'].isnull(), 'detail_title'].apply(
    lambda x: [])
df['detail_title'] = df['detail_title'].apply(lambda x: " ".join(x))

text_df = pd.DataFrame(df[EU_TIMELINE_TEXT_COLUMNS])
text_df.replace(np.nan, '', regex=True,inplace=True)
text_df['text'] = text_df.agg(' '.join, axis=1)
text_df.reset_index(drop=True,inplace=True)
text_df["emb"] = text_df["text"].apply(lambda x: text_to_vector(x,l2v_dict))

## Enrich dataset with new columns

In [5]:
for new_column in EU_TIMELINE_NEW_COLUMNS:
    model = get_best_model_from_ml_flow(experiment_ids=["1"], class_name=new_column)
    text_df[new_column] = model.predict(list(text_df["emb"]))


## Show result

In [6]:
text_df[['title']+EU_TIMELINE_NEW_COLUMNS[:]]

,title,businesses,citizens,workers
0,EU countries adopt guidelines on proof of vacc...,0,0,0
1,Commission unveils EU vaccines strategy,0,0,0
2,European roadmap shows path towards common lif...,0,0,0
3,Commission puts forward rules on rapid antigen...,0,0,0
4,EU leaders agree on the recovery plan and mult...,1,0,0
...,...,...,...,...
166,EUR 117 million for treatments and diagnostics...,1,0,0
167,€530 million to support emergency measures aga...,0,0,0
168,Setting out coordinated response to counter th...,0,0,0
169,Prolonging State aid Temporary Framework to fu...,0,0,0


## Analyse results

In [7]:
for new_column in EU_TIMELINE_NEW_COLUMNS:
    n_poz = len(text_df[text_df[new_column]==1])
    n_total = len(text_df)
    n_neg = n_total - n_poz
    print(f"{new_column}: n_neg = {round(n_neg/n_total,2)}, n_poz = {round(n_poz/n_total,2)}")

businesses: n_neg = 0.77, n_poz = 0.23
citizens: n_neg = 1.0, n_poz = 0.0
workers: n_neg = 0.95, n_poz = 0.05
